In [2]:
""" This is a controlled ADD/SUB that performs a,b -> a, a-b """
#NOTE this version costs nCNOT fewer than b-a

def ctrl_ADDSUB(qc, a, b, ctrl_value, c):
    """
    Takes in a quantum circuit and two quantum registers (a and b),
    performs an addition or subtraction, and returns the updated circuit.
    It dynamically creates its own carry qubits and removes them 
    before returning the updated circuit. Removals correspond to 
    measurement after the un-compute returns it to 0.

    #Requires n-1 ancillaries
    
    Parameters:
    - qc: QuantumCircuit on which the addition is performed
    - a: QuantumRegister for the first number
    - b: QuantumRegister for the second number
    - ctrl: Control bit conditioning whether a ADD or SUB is performed
    - c: QuantumRegister for carry qubits
    
    Returns:
    - QuantumCircuit: The updated quantum circuit with the addition operation applied.
    """

    n = len(a)  # number of qubits per register

    #a flip for subtraction
    for i in a:
        qc.cx(ctrl_value, i)
    
    qc.cx(ctrl_value,a[0])
    qc.cx(ctrl_value,b[0])
    qc.ccx(a[0], b[0], c[0]) 
    qc.cx(ctrl_value,c[0])

    #apply repeated segments
    for i in range(1,n-1):
        qc.cx(c[i-1],a[i])
        qc.cx(c[i-1],b[i])
        qc.ccx(a[i], b[i], c[i])
        qc.cx(c[i-1],c[i])
        
    #end segment (we have no carry-out in this implementation)
    qc.cx(c[n-2],b[n-1]) # n-1 as e.g. n = 4 final position is index=3

    #un compute ancillas
    for i in range(n-2,0,-1): # i goes from n-2 to 1
        qc.cx(c[i-1],c[i])
        qc.ccx(a[i], b[i], c[i])
        qc.cx(c[i-1],a[i])

    #uncompute c0 and complete the operation
    qc.cx(ctrl_value,c[0])
    qc.ccx(a[0], b[0], c[0])
    qc.cx(ctrl_value,a[0])

    for i in range(0,n):
        qc.cx(a[i],b[i])
    
    #need for b-a as a remains flipped (for a-b) the result is in b so no need to undo.
    for i in a:
        qc.cx(ctrl_value, i)

    
    return qc

In [1]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer

def init_random_num(qc, reg):
    """Randomly initializes an n-qubit quantum register and prints the register label and the binary number."""
    n = len(reg)  # number of qubits in the register
    binary_representation = ''
    
    for qubit in range(n-1): # choose to leave MSB as 0
        # Randomly apply X gate to flip the qubit with 50% probability
        if np.random.rand() > 0.5:
            qc.x(reg[qubit])  # Apply X gate to the quantum circuit
            binary_representation = '1' + binary_representation
        else:
            binary_representation = '0' + binary_representation
            
    binary_representation = '0' + binary_representation
    decimal_value = int(binary_representation, 2)
    
    # Print the label and the binary number created
    print(f"{reg.name}: {binary_representation} (decimal: {decimal_value})") # choose to leave MSB as 0
    
    return qc


In [7]:
""" MAIN, This does b - a"""

# Number of bits
n = 4

ancillaries = QuantumRegister(n-1, 'ancillaries')
ctrl = QuantumRegister(1, 'ctrl')
a = QuantumRegister(n, 'a')
b = QuantumRegister(n, 'b')

# Define a classical register with 2n bits (n for a and n for b)
result = ClassicalRegister(2 * n + 2*n, 'result')

qc = QuantumCircuit(a, b, ctrl, result, ancillaries)

"""initialize the quantum numbers"""
qc = init_random_num(qc, a)
qc = init_random_num(qc, b)

# Initialize the ctrl
SUB = True

if SUB == True:
    qc.x(ctrl[0]) #control 0 - add, 1 - sub

#apply the ctrl add/sub operation
qc = ctrl_ADDSUB(qc, a, b, ctrl[0], ancillaries)

# Measure the sum qubits and carry qubit
#qc.measure_all()

# Measure the qubits into the classical register
qc.measure(a, result[:n])      # Measure 'a' into the first n classical bits
qc.measure(b, result[n:2*n])   # Measure 'b' into the last n classical bits


# For execution
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
sim_result = simulator.run(compiled_circuit).result()
counts = sim_result.get_counts()

print(counts)

print("Measurement results:")
for bitstring, count in counts.items():
    # bitstring is of the form 'result' where the first n bits are for 'a' and the last n bits are for 'b'
    a_result = bitstring[-n:]  # Last n bits for b
    b_result = bitstring[-2*n:-n]  # First n bits for a
    a_decimal_value = int(a_result, 2)
    
    # Get the correct decimal value for b
    if SUB == True and b_result[0] == '1':
        b_decimal_value = -1 * (2**(n-1))
        for i in range(1, n):
            b_decimal_value += int(b_result[i]) * (2**(n-1-i))
    else:
        b_decimal_value = int(b_result, 2)
        
    print(f"a = {a_result} (decimal: {a_decimal_value}), b = {b_result} (decimal: {b_decimal_value})")

# Draw the circuit (we can't draw the circuit if we remove the qubits)
#qc_drawn = circuit_drawer(qc, output='text')
#print(qc_drawn)


a: 0000 (decimal: 0)
b: 0111 (decimal: 7)
{'0000000001110000': 1024}
Measurement results:
a = 0000 (decimal: 0), b = 0111 (decimal: 7)
